In [20]:
import pandas as pd

# ---------------------------------------------------------------------
# 1. Cargar los CSV desde tu carpeta local
# ---------------------------------------------------------------------
indic   = pd.read_csv("DATOS_TABLEROS/indicadores_vacunacion_municipal.csv",  dtype=str)
vac_dep = pd.read_csv("DATOS_TABLEROS/vacunacion_departamental_2009_2010.csv", dtype=str)
vac_mun = pd.read_csv("DATOS_TABLEROS/vacunacion_municipal.csv",            dtype=str)

# ---------------------------------------------------------------------
# 1-bis. Limpiar: mayúsculas en columnas + quitar espacios EN LAS CELDAS
# ---------------------------------------------------------------------
for d in (indic, vac_dep, vac_mun):
    d.columns = d.columns.str.strip().str.upper()          # columnas
    for c in d.select_dtypes("object"):                    # celdas
        d[c] = d[c].str.strip()
        
    d["ANO"] = d["ANO"].astype(int)

# ---------------------------------------------------------------------
# 2. DIM_FECHA
# ---------------------------------------------------------------------
años = (
    pd.concat([indic["ANO"], vac_dep["ANO"], vac_mun["ANO"]])
      .astype(int).unique()
)
dim_fecha = pd.DataFrame({"ANO": sorted(años)})
dim_fecha["FECHA_ID"] = dim_fecha.index + 1        # 1, 2, 3…

# ---------------------------------------------------------------------
# 3. DIM_UBICACION
# ---------------------------------------------------------------------
def loc_cols(df):
    return df[["DEPARTAMENTO", "CODIGO MUNICIPIO", "MUNICIPIO"]]

loc_mun = pd.concat([loc_cols(vac_mun), loc_cols(indic)], ignore_index=True)

loc_dep = vac_dep[["DEPARTAMENTO"]].drop_duplicates()
loc_dep["CODIGO MUNICIPIO"] = "0"      # dummy
loc_dep["MUNICIPIO"]        = None

dim_ubicacion = (
    pd.concat([loc_mun, loc_dep], ignore_index=True)
      .drop_duplicates()
      .reset_index(drop=True)
)
dim_ubicacion["UBICACION_ID"] = dim_ubicacion.index + 1

# ---------------------------------------------------------------------
# 4. HECHO CONTEXTO SOCIOECONÓMICO
# ---------------------------------------------------------------------
hecho_contexto = (
    indic
      .merge(dim_fecha,     on="ANO")
      .merge(dim_ubicacion, on=["DEPARTAMENTO", "CODIGO MUNICIPIO", "MUNICIPIO"])
      [["FECHA_ID", "UBICACION_ID",
        "IPM_POBREZA", "IPM_SIN_SALUD", "IPM_SIN_ACCESO_SALUD",
        "NIVEL_SISBEN", "PROPORCION_RURAL", "EDUCACIÓN_NETA_TOTAL",
        "INGRESOS TOTALES", "PCTE_POBLACION_RURAL"]]
)

# ---------------------------------------------------------------------
# 5. HECHO JORNADA VACUNACIÓN
# ---------------------------------------------------------------------
vac_dep2 = vac_dep.copy()
vac_dep2["CODIGO MUNICIPIO"] = "0"
vac_dep2["MUNICIPIO"]        = None

vac_all = pd.concat([vac_mun, vac_dep2], ignore_index=True)

hecho_vacunacion = (
    vac_all
      .merge(dim_fecha,     on="ANO")
      .merge(dim_ubicacion, on=["DEPARTAMENTO", "CODIGO MUNICIPIO", "MUNICIPIO"])
      [["FECHA_ID", "UBICACION_ID", "META", "DOSIS APLICADAS", "COBERTURA"]]
)



In [21]:
dim_fecha

,ANO,FECHA_ID
0,2009,1
1,2010,2
2,2022,3
3,2023,4


In [22]:
dim_ubicacion

,DEPARTAMENTO,CODIGO MUNICIPIO,MUNICIPIO,UBICACION_ID
0,ANTIOQUIA,5002.0,ABEJORRAL,1
1,ANTIOQUIA,5004.0,ABRIAQUI,2
2,ANTIOQUIA,5021.0,ALEJANDRIA,3
3,ANTIOQUIA,5030.0,AMAGA,4
4,ANTIOQUIA,5031.0,AMALFI,5
...,...,...,...,...
700,SUCRE,0,None,701
701,TOLIMA,0,None,702
702,VALLE,0,None,703
703,VAUPÉS,0,None,704


In [23]:
hecho_contexto

,FECHA_ID,UBICACION_ID,IPM_POBREZA,IPM_SIN_SALUD,IPM_SIN_ACCESO_SALUD,NIVEL_SISBEN,PROPORCION_RURAL,EDUCACIÓN_NETA_TOTAL,INGRESOS TOTALES,PCTE_POBLACION_RURAL
0,1,335,0,0,0,0,0,0.89,8193.65,0
1,1,336,0,0,0,0,0,0.88,2992.01,0
2,1,337,0,0,0,0,0,1.13,5219.18,0
3,1,338,0,0,0,0,0,0.81,11334.66,0
4,1,339,0,0,0,0,0,0.88,17377.29,0
...,...,...,...,...,...,...,...,...,...,...
1026,4,629,0,0,0,0,0,0,0,0.4
1027,4,630,0,0,0,0,0,0,0,0.3599
1028,4,631,0,0,0,0,0,0,0,0.4649
1029,4,632,0,0,0,0,0,0,0,0.1322


In [24]:
hecho_vacunacion

,FECHA_ID,UBICACION_ID,META,DOSIS APLICADAS,COBERTURA
0,1,1,364,300,82.4
1,1,2,40,28,70.0
2,1,3,51,53,103.9
3,1,4,558,380,68.1
4,1,5,480,413,86.0
...,...,...,...,...,...
1098,2,701,16873,16351.0,96.9
1099,2,702,25717,22892.0,89.0
1100,2,703,72781,69350.0,95.3
1101,2,704,1230,902.0,73.3


In [26]:
# ---------------------------------------------------------------------
# 6. Exportar a CSV (para Power BI)
# ---------------------------------------------------------------------
dim_fecha.to_csv("MODELO/dim_fecha.csv", index=False)
dim_ubicacion.to_csv("MODELO/dim_ubicacion.csv", index=False)
hecho_contexto.to_csv("MODELO/hecho_contexto_socioeconomico.csv", index=False)
hecho_vacunacion.to_csv("MODELO/hecho_jornada_vacunacion.csv", index=False)